# Cálculo de deducciones incrementadas de Tramo 1 y Tramo 2

## A. Importación de módulos y librerías

In [ ]:
import pandas as pd
import numpy as np

from remuneraciones.prorrateos import *
from parametrizacion.conceptos import ParamConceptos
from deducciones.incremental.tablas_tramo2 import *

from helpers.bases import tabla
from helpers.progressbar import ProgressBar
from helpers.dict import unpack

from data import CWTR_SUM, DOTACION, CUILES, F572_WEB

## B. Parametrización de conceptos

In [ ]:
conceptos = ParamConceptos()
conceptos

## C. Deducción incrementada Tramo 1

### 1. Unión de los orígenes de datos

#### CWTR

In [ ]:
cwtr = CWTR_SUM[['legajo', 'CCn', 'mes', 'importe']].copy()
cwtr.rename(columns={'CCn': 'codigo'}, inplace=True)

cwtr

#### Prorrateos

In [ ]:
legajos_meses_tramo1 = tabla('bases_tramos').query('tramo == 1')
prorrateados = Prorrateos().prorrateados()

prorrateos_aplicables = prorrateados.merge(legajos_meses_tramo1, how='inner', left_on=['legajo', 'mes_pago'], right_on=['legajo', 'mes'])
prorrateos = prorrateos_aplicables[['legajo', 'CCn', 'mes_x', 'importe_x']].copy()
prorrateos.rename(columns={'CCn': 'codigo', 'mes_x': 'mes', 'importe_x': 'importe'}, inplace=True)

prorrateos

#### F.572 Web

In [ ]:
f572 = F572_WEB.copy()
f572['cuil'] = f572['cuil'].astype('int64')
f572 = f572.merge(CUILES, how='left', on='cuil')
f572 = f572[['legajo', 'codigo', 'mes', 'importe']]
f572.dropna(inplace=True)

f572

##### Topes aplicables

In [ ]:
deducciones_con_tope = {

    'fijo': {
        'Q-Deduc-22':	252_564.84,
        'Q-Deduc-24':	42_921.24,
        'Q-Deduc-8':	252_564.84,
        'Q-Deduc-32':	101_025.94,
        'Q-Deduc-4':	20_000,
        'Q-Deduc-23':	42_921.24,
        'Q-Deduc-2':	42_921.24,
    },

    'variable': [
        'Q-Deduc-1',
        'Q-Deduc-3',
        'Q-Deduc-7',
    ]
}


cod_tope_fijo = list(deducciones_con_tope['fijo'].keys())
cod_tope_var = deducciones_con_tope['variable']

cod_tope = cod_tope_fijo.copy()   #extend(cod_tope_var)
cod_tope.extend(cod_tope_var)



df_topes_fijos = []

for cod, tope in deducciones_con_tope['fijo'].items():
    df_topes_fijos.append({'codigo': cod, 'tope_anual': tope})

df_topes_fijos = pd.DataFrame.from_records(df_topes_fijos)
df_topes_fijos

##### SIN tope

In [ ]:
f572_sin_tope = f572[~f572['codigo'].isin(cod_tope)]


##### CON tope (fijo)

In [ ]:
deduc_tope_fijo = f572[f572['codigo'].isin(cod_tope_fijo)]
deduc_tope_fijo = deduc_tope_fijo.groupby(['legajo','codigo','mes'])['importe'].sum().reset_index()


In [ ]:
df = deduc_tope_fijo.merge(df_topes_fijos, how='left', on='codigo')

df['tope_aplicable'] = np.where(df['mes'].eq(13), df['tope_anual'], df['tope_anual'] / 12)

df['topeado'] = df[['importe','tope_aplicable']].min(axis=1)
df = df[['legajo', 'codigo', 'mes', 'topeado']]
f572_tope_fijo = df.rename(columns={'topeado': 'importe'})
f572_tope_fijo.to_clipboard()

##### CON tope (variable)

In [ ]:
subtotal_conceptos = pd.concat([cwtr, f572_sin_tope, f572_tope_fijo, prorrateos])

df = subtotal_conceptos
df['legajo'] = df['legajo'].astype('int64')

df['Suma'] = np.where(df['codigo'].isin(conceptos.listar('TopeVarDI', '+')), df['importe'], 0)
df['Resta'] = np.where(df['codigo'].isin(conceptos.listar('TopeVarDI', '-')), df['importe'], 0)
df['GanNetaTope5'] = df['Suma'] - df['Resta']
df['tope5'] = df['GanNetaTope5'] * 0.05

df = df.groupby(['legajo','mes'])['tope5'].sum().reset_index()
df['cero'] = 0
df['tope5'] = df[['tope5','cero']].max(axis=1)

tope_5_mensual = df[['legajo', 'mes', 'tope5']]
tope_5_mensual

In [ ]:
deduc_tope_var = f572[f572['codigo'].isin(cod_tope_var)]

deduc_tope_var = deduc_tope_var.merge(tope_5_mensual, how='left', on=['legajo', 'mes'])
deduc_tope_var

In [ ]:
df = deduc_tope_var
df['topeado'] = df[['importe','tope5']].min(axis=1)
df = df[['legajo', 'codigo', 'mes', 'topeado']]
f572_tope_variable = df.rename(columns={'topeado': 'importe'})
f572_tope_variable

#### Deducciones de Ley

In [ ]:
minimo_no_imponible = {
    'normal': {
        'mni': 252_564.84,
        'esp': 1_212_311.24
    },
    'especial': {
        'mni': 308_129.12,
        'esp': 1_479_019.72
    }
}

provincias_especiales = [
    'Neuquén',
    'Chubut',
    'Río Negro',
    'Santa Cruz',
    'La Pampa',
]

In [ ]:
from data import CWTR

cwtr_provincias = CWTR[['legajo', 'mes', 'provincia']].drop_duplicates().reset_index()

prov_dotacion = list(cwtr_provincias['provincia'].unique())

for p in provincias_especiales:
    if p not in prov_dotacion:
        raise NotImplementedError(f'No hay empleados en la provincia: {p}')

cwtr_provincias

In [ ]:
deducciones_ley = []

for i, row in cwtr_provincias.iterrows():

    legajo, mes, provincia = unpack(row, 'legajo', 'mes', 'provincia')

    tipo_mni = 'especial' if provincia in provincias_especiales else 'normal'
    deducciones = minimo_no_imponible[tipo_mni]

    deducciones_ley.append({'legajo': legajo, 'codigo': 'DedLey-MNI', 'mes': mes, 'importe': deducciones['mni'] / 12 })
    deducciones_ley.append({'legajo': legajo, 'codigo': 'DedLey-Especial', 'mes': mes, 'importe': deducciones['esp'] / 12 })

deducciones_ley = pd.DataFrame.from_records(deducciones_ley)
deducciones_ley

#### Unión

In [ ]:
total_conceptos = pd.concat([cwtr, f572_sin_tope, f572_tope_fijo, f572_tope_variable, prorrateos, deducciones_ley])
total_conceptos

### 2. Filtrado de conceptos aplicables

In [ ]:
# Control
conceptos_parametrizados = list(conceptos.data['CCn'].unique())

for c in list(total_conceptos['codigo'].unique()):
    if c not in conceptos_parametrizados:
        raise NotImplementedError(f'No se ha definido parametría para el concepto: {c}')

In [ ]:
df = total_conceptos
df['legajo'] = df['legajo'].astype('int64')

df['Suma'] = np.where(df['codigo'].isin(conceptos.listar('DedIncT1', '+')), df['importe'], 0)
df['Resta'] = np.where(df['codigo'].isin(conceptos.listar('DedIncT1', '-')), df['importe'], 0)
df['DeduccIncremT1'] = df['Suma'] - df['Resta']

# Se filtran cuando sólo sean tramo 1
df = df.merge(legajos_meses_tramo1, how='inner', on=['legajo', 'mes'])

In [ ]:
deduc_incr_tramo1_mensual = df.groupby(['legajo','mes'])['DeduccIncremT1'].sum().reset_index()
deduc_incr_tramo1_anual = df.groupby(['legajo'])['DeduccIncremT1'].sum().reset_index()

deduc_incr_tramo1_mensual.to_excel('prueba.xlsx')

## D. Deducción incrementada Tramo 2

In [ ]:
legajos_meses_tramo2 = tabla('bases_tramos').query('tramo == 2')
legajos_meses_tramo2

In [ ]:
tabla('tablas_afip_tramo2')

In [ ]:
deducc_tramo2 = []

p = ProgressBar(len(legajos_meses_tramo2))
for i, row in legajos_meses_tramo2.iterrows():
    
    legajo, base, mes = unpack(row, 'legajo', 'base_tramo', 'mes')

    deduccion = TablasTramo2.deduccion(mes, base)
    deducc_tramo2.append({
        'legajo':           legajo, 
        'mes':              mes, 
        'DeduccIncremT2':   deduccion
    })
    p.next()

deducc_tramo2 = pd.DataFrame.from_records(deducc_tramo2)
deducc_tramo2

## E. Unificación con base tramos

In [ ]:
con_tramo1 = tabla('bases_tramos').merge(deduc_incr_tramo1_mensual, how='left', on=['legajo', 'mes'])
con_tramo2 = con_tramo1.merge(deducc_tramo2, how='left', on=['legajo', 'mes'])

In [ ]:
con_tramo2.to_excel('deducciones_incrementadas.xlsx')